In [2]:
import pandas as pd
import json
from IPython.display import display
import os
import geopandas as gpd
from geopandas.testing import assert_geodataframe_equal
from shapely import wkt

# Atrybuty z OSM - złączenie USA i PL

In [6]:
ny = pd.read_csv("../data/downstream_tasks/airbnb/new_york/New York City_all_tags_one_hot_encoded.csv", nrows=2)
ny = ny.drop(["osmid", 'geometry'], axis=1)
ny

,aeroway_aerodrome,aeroway_apron,aeroway_gate,aeroway_hangar,aeroway_helipad,aeroway_heliport,aeroway_holding_position,aeroway_jet_bridge,aeroway_navigationaid,aeroway_parking_position,...,waterway_fuel,waterway_link,waterway_lock_gate,waterway_put_in,waterway_river,waterway_sluice_gate,waterway_stream,waterway_tidal_channel,waterway_waterfall,waterway_weir
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
st = pd.read_csv("../data/downstream_tasks/airbnb/seattle/Seattle_all_tags_one_hot_encoded.csv", nrows=2)
st = st.drop(["osmid", 'geometry'], axis=1)
st

,feature_id,aeroway_aerodrome,aeroway_apron,aeroway_hangar,aeroway_helipad,aeroway_holding_position,aeroway_navigationaid,aeroway_parking_position,aeroway_runway,aeroway_service,...,waterway_ditch,waterway_dock,waterway_drain,waterway_fish_pass,waterway_lock_gate,waterway_river,waterway_slough,waterway_stream,waterway_waterfall,waterway_yes
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
nycols = list(ny.columns)

In [9]:
len(nycols + list(st.columns))

1885

In [10]:
cities_pol = ["poznan", "wroclaw", "warszawa", "krakow", "szczecin"]
all_cols = []
for city in cities_pol:
    df = pd.read_parquet(f"../data/organized-datasets/{city}/2022/h9/count-embedder/dataset.parquet")
    df = df.drop(["region_id", "geometry"], axis=1)
    city_cols = df.columns
    all_cols.extend(city_cols)

In [11]:
all_cols.extend(nycols)
all_cols.extend(list(st.columns))

In [12]:
len(all_cols)

6063

In [13]:
len(set(all_cols))

2152

In [14]:
all_attrs = {}
all_attrs["cols"] = list(set(all_cols))
with open('../data/hex_attributes.json', 'w') as f:
    json.dump(all_attrs, f)

In [15]:
with open('../data/hex_attributes.json', 'r') as file:
        all_attrs = json.load(file)

In [16]:
all_columns = all_attrs['cols']

In [17]:
len(all_columns)

2152

# Dodanie kolumn zgodnie z tagami

## Count embedder

In [18]:
cities_pol = ["poznan", "wroclaw", "warszawa", "krakow", "szczecin"]
year=2022
for city in cities_pol:
    gdf = gpd.read_parquet(f"../data/organized-datasets/{city}/2022/h9/count-embedder/dataset.parquet")
    for col in all_columns:
        if col not in gdf.columns:
            gdf[col] = 0
    display(gdf)
    if not os.path.exists(f"../data/organized-hexes/{city}/{year}/h9/count-embedder/"):
        os.makedirs(f"../data/organized-hexes/{city}/{year}/h9/count-embedder/")
    gdf.to_parquet(f"../data/organized-hexes/{city}/{year}/h9/count-embedder/dataset.parquet")
    gdf_test = gpd.read_parquet(f"../data/organized-hexes/{city}/{year}/h9/count-embedder/dataset.parquet")

    assert_geodataframe_equal(gdf, gdf_test, check_like=True)


/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

,region_id,aeroway_aerodrome,aeroway_apron,aeroway_gate,aeroway_hangar,aeroway_helipad,aeroway_holding_position,aeroway_parking_position,aeroway_runway,aeroway_stopway,...,leisure_farm,amenity_prep_school,shop_eggs,historic_heritage,aeroway_navigationaid,historic_park,historic_train_station,shop_hobby,building_floating_home,building_castle
0,891e24aa333ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,891e24a338bffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,891e24aa667ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,891e24aa063ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,891e24b838fffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2940,891e24a803bffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2941,891e24aa4d7ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2942,891e24a85bbffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2943,891e24ab58bffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

,region_id,aeroway_aerodrome,aeroway_apron,aeroway_gate,aeroway_hangar,aeroway_helipad,aeroway_holding_position,aeroway_parking_position,aeroway_runway,aeroway_taxiway,...,waterway_stream_end,shop_flag,leisure_sport,leisure_farm,amenity_prep_school,historic_heritage,aeroway_navigationaid,historic_park,shop_hobby,building_floating_home
0,891e204e4a7ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,891e204239bffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,891e2041837ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,891e2047223ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,891e2046657ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,891e2041137ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3164,891e2042273ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3165,891e20430d7ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3166,891e2050a13ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

,region_id,aeroway_aerodrome,aeroway_apron,aeroway_gate,aeroway_helipad,aeroway_holding_position,aeroway_lounge,aeroway_navigationaid,aeroway_parking_position,aeroway_rc,...,historic_quarry,amenity_kick_scooter_parking,sport_four_square,building_gate,shop_linen,waterway_stream_end,leisure_farm,shop_eggs,historic_train_station,building_floating_home
0,891f53ca5cfffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,891f522611bffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,891f5352e6fffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,891f535046fffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,891f53ca6a3ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5570,891f53c160bffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5571,891f522418fffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5572,891f5352b43ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5573,891f53504afffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

,region_id,aeroway_aerodrome,aeroway_hangar,aeroway_helipad,aeroway_parking_position,aeroway_runway,aeroway_windsock,amenity_animal_boarding,amenity_animal_shelter,amenity_arcade,...,sport_four_square,shop_engraver,building_gate,shop_linen,waterway_stream_end,shop_flag,leisure_farm,aeroway_navigationaid,historic_train_station,building_floating_home
0,891e2e698c3ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,891e2e79abbffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,891e2e693d7ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,891e2e6b09bffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,891e2e79b13ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3412,891e2e68137ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3413,891e2e61b77ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3414,891e05a64b3ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3415,891e2e7994fffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

,region_id,aeroway_aerodrome,aeroway_helipad,aeroway_runway,aeroway_taxiway,aeroway_windsock,amenity_accounting_office,amenity_animal_breeding,amenity_animal_shelter,amenity_arts_centre,...,leisure_sport,leisure_farm,shop_eggs,historic_heritage,aeroway_navigationaid,historic_park,historic_train_station,shop_hobby,building_floating_home,building_castle
0,891f0e7b2d3ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,891f0e78e8fffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,891f0e79357ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,891f0e7937bffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,891f0e4e937ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3529,891f1db6a6bffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3530,891f0e796cfffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3531,891f0e7a9d3ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3532,891f1db69b7ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
cities_usa = ["New_York"]
year=2024
for city in cities_usa:
    df = pd.read_csv(f"../data/organized-datasets/organized_USA/{city}_res_9_features.csv")
    for col in all_columns:
        if col not in df.columns:
            df[col] = 0
    df_geom = pd.read_csv(f"../data/downstream_tasks/airbnb/new_york/New York City_res_9.csv")
    merged = df.merge(df_geom, on="region_id", how='inner')
    display(merged)
    
    merged['geometry'] = merged['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(merged, geometry='geometry')
    if not os.path.exists(f"../data/organized-hexes/new_york/{year}/h9/count-embedder/"):
        os.makedirs(f"../data/organized-hexes/new_york/{year}/h9/count-embedder/")
    gdf.to_parquet(f"../data/organized-hexes/new_york/{year}/h9/count-embedder/dataset.parquet")
    gdf_test = gpd.read_parquet(f"../data/organized-hexes/new_york/{year}/h9/count-embedder/dataset.parquet")
    assert_geodataframe_equal(gdf, gdf_test, check_like=True)

/tmp/ipykernel_117161/1895354731.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = 0
/tmp/ipykernel_117161/1895354731.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = 0
/tmp/ipykernel_117161/1895354731.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = 0
/tmp/ipyke

,region_id,aeroway_aerodrome,aeroway_apron,aeroway_gate,aeroway_hangar,aeroway_helipad,aeroway_heliport,aeroway_holding_position,aeroway_jet_bridge,aeroway_navigationaid,...,waterway_stream_end,shop_flag,leisure_farm,shop_eggs,historic_park,historic_train_station,building_floating_home,amenity_vacuum_cleaner,building_castle,geometry
0,892a1000003ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POLYGON ((-73.78198948843054 40.86017921842936...
1,892a1000007ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POLYGON ((-73.77975448365349 40.85758861169454...
2,892a100000bffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POLYGON ((-73.78627525032026 40.86007112650341...
3,892a100000fffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((-73.78404000610331 40.857480601484, ..."
4,892a1000013ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POLYGON ((-73.77993873829112 40.86287788509053...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10752,892a10776cbffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POLYGON ((-74.02132692596481 40.67672226431196...
10753,892a10776cfffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POLYGON ((-74.01908705622313 40.67414212649311...
10754,892a10776d3ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((-74.01502796991744 40.679536127653, ..."
10755,892a10776d7ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POLYGON ((-74.01278832283833 40.67695578075376...


In [21]:
cities_usa = ["Seattle"]
year=2024
for city in cities_usa:
    df = pd.read_csv(f"../data/organized-datasets/organized_USA/{city}_res_9_features.csv")
    for col in all_columns:
        if col not in df.columns:
            df[col] = 0
    df_geom = pd.read_csv(f'../data/downstream_tasks/airbnb/seattle/Seattle_res_9.csv')
    merged = df.merge(df_geom, on="region_id", how='inner')
    display(merged)
    merged['geometry'] = merged['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(merged, geometry='geometry')
    if not os.path.exists(f"../data/organized-hexes/seattle/{year}/h9/count-embedder/"):
        os.makedirs(f"../data/organized-hexes/seattle/{year}/h9/count-embedder/")
    gdf.to_parquet(f"../data/organized-hexes/seattle/{year}/h9/count-embedder/dataset.parquet")
    gdf_test = gpd.read_parquet(f"../data/organized-hexes/seattle/{year}/h9/count-embedder/dataset.parquet")
    assert_geodataframe_equal(gdf, gdf_test, check_like=True)

/tmp/ipykernel_117161/1431666553.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = 0
/tmp/ipykernel_117161/1431666553.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = 0
/tmp/ipykernel_117161/1431666553.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = 0
/tmp/ipyke

,region_id,aeroway_aerodrome,aeroway_apron,aeroway_hangar,aeroway_helipad,aeroway_holding_position,aeroway_navigationaid,aeroway_parking_position,aeroway_runway,aeroway_service,...,shop_linen,waterway_stream_end,shop_flag,leisure_sport,shop_eggs,historic_park,historic_train_station,shop_hobby,building_castle,geometry
0,8928d540003ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POLYGON ((-122.25110252634171 47.6504267090504...
1,8928d540007ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POLYGON ((-122.2550930308571 47.65151394545429...
2,8928d54000bffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POLYGON ((-122.24731559220862 47.6521925423558...
3,8928d54000fffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POLYGON ((-122.25130621861749 47.6532798738718...
4,8928d540013ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POLYGON ((-122.2508988476786 47.64757342279738...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4198,8928d5cdbcbffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POLYGON ((-122.39215187876377 47.4929055470186...
4199,8928d5cdbcfffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POLYGON ((-122.39613348154163 47.4939917364338...
4200,8928d5cdbd3ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POLYGON ((-122.39571739712908 47.4882749082355...
4201,8928d5cdbd7ffff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POLYGON ((-122.39969874427203 47.4893610887909...
